In [ ]:
from osgeo import gdal, osr
import os
import numpy as np
import pandas as pd
import time
import glob
import datetime as dt
# from numba import jit
# import xarray as xr

In [ ]:
# hdf_path=r"H:\C6.1\AQUA\2002\data\A2002185\MYD04_L2.A2002185.0425.061.2018003214418.hdf"

# hdf_dataset=xr.open_dataset(hdf_path,engine='netcdf4')
# lon_arr=hdf_dataset['Longitude'].values
# lat_arr=hdf_dataset['Latitude'].values
# Solar_Zenith_arr=hdf_dataset['Solar_Zenith'].values
# Average_Cloud_Pixel_Distance_Land_Ocean_arr=hdf_dataset['Average_Cloud_Pixel_Distance_Land_Ocean'].values

In [ ]:
def swath_georeference(in_file, target_dataset_name, out_file, x_res_out, y_res_out,out_geo_range=None):
    '''
    - `in_file`：输入文件的路径，这是一个包含卫星轨道数据的文件。
    - `target_dataset_name`：目标数据集的名称，这可是输入文件中的一个特定子集或层。
    - `out_file`：输出文件的路径，这是一个包含地理参考后的数据的新文件。
    - `x_res_out` 和 `y_res_out`：输出数据的空间分辨率，这决定了输出图像的每个像素代表的地理区域的大小。
    - `out_geo_range`：输出数据的地理范围，这是一个可选参数，如果提供，输出数据将被裁剪到这个范围。
    - `swath_georeference` 函数的目的是将卫星轨道数据转换为地理参考的数据，这样每个像素都可以对应到地球上的一个具体位置。
    '''    
    dataset = gdal.Open(in_file)
    if dataset == None:
        print('{} is not found or open.'.format(in_file))#转换失败
        return False
    subdatasets = dataset.GetSubDatasets()
    dataset_pos = 0
    target_pos = -1
    #找到目标数据集并返回索引号
    for subdataset in subdatasets:
        subdataset_name = subdataset[0]
        if subdataset_name.endswith(target_dataset_name):
            target_pos = dataset_pos
        dataset_pos += 1
    if target_pos != -1:
        target_dataset = dataset.GetSubDatasets()[target_pos][0]
        vrt_file = os.path.splitext(in_file)[0] + '.vrt'
        vrt_out = gdal.Translate(vrt_file, target_dataset,format='vrt', unscale='true')
        sr = osr.SpatialReference()
        sr.SetWellKnownGeogCS('EPSG:4326')#WGS-84坐标EPSG:4326
        georeferenced_data = gdal.Warp(out_file, vrt_out, multithread=True, outputBounds=out_geo_range,
                                       format='GTiff', geoloc=True, dstSRS=sr.ExportToWkt(),
                                       xRes=x_res_out, yRes=y_res_out, srcNodata=-9999,dstNodata=-9999,resampleAlg=gdal.GRIORA_NearestNeighbour,outputType=gdal.GDT_Float32,
                                       options=["TILED=YES", "COMPRESS=LZW"])
        if georeferenced_data == None:
            print('{}  is False.'.format(in_file))#转换失败
            os.remove(vrt_file)
        else:
            # print('The georeference of {} is complete.'.format(in_file))#转换成功
            # del_errorValue(out_file)#删除异常值
            os.remove(vrt_file)
            pass
    else:
        print('{} has no the target dataset.'.format(in_file))
    # return True
    

In [ ]:
# hdf=r"H:\C6.1\AQUA\2013\data\A2013001\MYD04_L2.A2013001.0505.061.2018044005624.hdf"
# output_directory=r"E:\Orange\desktop\test"

# dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land_Best_Estimate'
# # dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land'
# geo_range = None  # 不定义图像范围
# x_res = 0.1  # 分辨率
# y_res = 0.1  # 分辨率

# # file_name = os.path.join(input_directory, file_i)
# out_file = os.path.join(output_directory, os.path.splitext(os.path.basename(hdf))[0] + '_geo_new.tif')
# swath_georeference(hdf, dataset_name,out_file,x_res, y_res,geo_range)


In [ ]:
# if __name__ == '__main__':
#     file_prefix = '.hdf'  # 文件后缀
#     input_directory = r'D:\C6.1\AQUA\2017\data'  # 输入文件
#     output_directory = r'D:\C6.1\AQUA\2017\Deep_Blue_Aerosol_Optical_Depth_550_Land'  # 输出文件
#     dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land'  # 所选取数据集的名称

#     if not os.path.exists(output_directory):
#         os.mkdir(output_directory)

#     file_list = os.listdir(input_directory)
#     for file_i in file_list:
#         if file_i.endswith(file_prefix):
#             file_name = os.path.join(input_directory, file_i)

#             out_file = os.path.join(
#                 output_directory, os.path.splitext(file_i)[0] + '_geo.tiff')
#             # dataset_name = 'Radiance'#辐射值
#             geo_range = None  # 不定义图像范围
#             x_res = 0.1  # 分辨率
#             y_res = 0.1  # 分辨率
#             swath_georeference(file_name, dataset_name,
#                                out_file, geo_range, x_res, y_res)
# print("ok")

In [ ]:
import os
import glob

input_directory = r'H:\C6.1\AQUA'  # 输入文件
output_directory = r'I:\MODIS_AQUA_DB_TIF\Aerosol_Type_Land'  # 输出文件
dataset_name = 'Average_Cloud_Pixel_Distance_Land_Ocean'  # 所选取数据集的名称

for year in range(2002, 2021):
    days_folder = os.path.join(input_directory, str(year), "data")
    for days in os.listdir(days_folder):
        tifs = glob.glob(os.path.join(days_folder, days, '*.hdf'))
        for tif in tifs:
            output_folder = os.path.join(output_directory, str(year), days)
            os.makedirs(output_folder, exist_ok=True)
            out_file = os.path.join(output_folder, os.path.splitext(
                os.path.basename(tif))[0] + '_geo.tif')
            if not os.path.exists(out_file):
                geo_range = None  # 不定义图像范围
                x_res = 0.1  # 约为500m分辨率
                y_res = 0.1  # 约为500m分辨率
                swath_georeference(tif, dataset_name, out_file, x_res, y_res,geo_range)
print("ok")

#Deep_Blue_Aerosol_Optical_Depth_550_Land
# H:\C6.1\AQUA\2018\data\A2018008\MYD04_L2.A2018008.0810.061.2018008194403.hdf is not found or open.
# H:\C6.1\AQUA\2018\data\A2018008\MYD04_L2.A2018008.0815.061.2018008195121.hdf is not found or open.

In [ ]:
# H:\C6.1\AQUA\2018\data\A2018008\MYD04_L2.A2018008.0810.061.2018008194403.hdf is not found or open.
# H:\C6.1\AQUA\2018\data\A2018008\MYD04_L2.A2018008.0815.061.2018008195121.hdf is not found or open.

In [ ]:
# import os
# from concurrent import futures

# file_prefix = '.hdf'  # 文件后缀
# input_directory = r'D:\C6.1\AQUA\2018\data'  # 输入文件
# output_directory = r'D:\C6.1\AQUA\2018\Deep_Blue_Aerosol_Optical_Depth_550_Land'  # 输出文件
# dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land'  # 所选取数据集的名称

# if not os.path.exists(output_directory):
#     os.mkdir(output_directory)

# file_list = os.listdir(input_directory)

# def process_file(file_i):
#     if file_i.endswith(file_prefix):
#         file_name = os.path.join(input_directory, file_i)
#         out_file = os.path.join(output_directory, os.path.splitext(file_i)[0] + '_geo.tiff')
#         geo_range = None  # 不定义图像范围
#         x_res = 0.1  # 约为500m分辨率
#         y_res = 0.1  # 约为500m分辨率
#         swath_georeference(file_name, dataset_name, out_file, geo_range, x_res, y_res)

# with futures.ThreadPoolExecutor() as executor:
#     executor.map(process_file, file_list)
#     print("ok")